# Setup packages

In [ ]:
include("setup.jl")

# RK22 series examples

This section of code reproduces the results in Sections 2.1.1 and 2.3 of the paper.

In [ ]:
α = sp.symbols("α", real=true)
μ = sp.symbols("μ", real=true)
A = [0 0; 1/(2*α) 0]; b = [1-α, α]; c = [0, 1/(2*α)]
A2 = [0 0; 1/(2*μ) 0]; b2 = [1-μ, μ]; c2 = [0, 1/(2*μ)]

coeffs = bseries(A,b,c,3)
coeffs2 = bseries(A2,b2,c2,3)

latexify(coeffs, cdot=false)

In [ ]:
comp = compose(coeffs,coeffs2,normalize_stepsize=true)
latexify(comp, cdot=false)

In [ ]:
println(latexify(comp, cdot=false))

# Modified equations (Lotka-Volterra)

This reproduces the example in Section 2.4.2.

In [ ]:
function f(du, u, params, t)
  p, q = u
  dp = (2 - q) * p
  dq = (p - 1) * q
  du[1] = dp; du[2] = dq
  return nothing
end

u0 = [1.5, 2.25]
tspan = (0.0, 15.0)
ode = ODEProblem(f, u0, tspan)

dt = 0.1
sol_euler = solve(ode, Euler(), dt=dt)
sol_ref = solve(ode, Tsit5(),saveat=0:0.1:15)

In [ ]:
plt.rc("lines", linewidth=2.5, markersize=5, markeredgewidth=2.5)
plt.rc("legend", loc="best", fontsize="small", fancybox=true, framealpha=1.0)

In [ ]:
u = sol_ref.u
plt.plot(getindex.(u,2),getindex.(u,1),color="k")
plt.scatter(last.(sol_euler.u),first.(sol_euler.u),color="#56B4E9")
plt.legend(["Reference solution",L"Explicit Euler, $\Delta t = 0.1$"]);
plt.xlabel(L"$q$"); plt.ylabel(L"$p$");
#plt.savefig("test.pdf")

In [ ]:
function solve_modified_equation(ode, truncation_orders, dt)
  # Explicit Euler method
  A = @SArray [0//1;]
  b = @SArray [1//1]
  c = @SArray [0//1]

  # Setup of symbolic variables
  @variables dt_sym
  u_sym = @variables p q
  f_sym = similar(u_sym); f(f_sym, u_sym, nothing, nothing)

  sol_euler = solve(ode, Euler(), dt=dt)

  plt.scatter(last.(sol_euler.u), first.(sol_euler.u),
              label="Explicit Euler, dt = $dt", color="k")

  for truncation_order in truncation_orders
    series = modified_equation(f_sym, u_sym, dt_sym, A, b, c, truncation_order)
    series = Symbolics.substitute.(series, dt_sym => dt)
    modified_f, _ = build_function(series, u_sym, expression=Val(false))
    modified_ode = ODEProblem((u, params, t) -> modified_f(u), ode.u0, tspan)
    modified_sol = solve(modified_ode, Tsit5(), abstol=1.0e-6, reltol=1.0e-6)
    fig = plt.plot(last.(modified_sol.u), first.(modified_sol.u),
                    label="Modified ODE, order $(truncation_order-1)")
  end
  plt.legend()
  plt.xlabel(L"$q$"); plt.ylabel(L"$p$");
end

fig = solve_modified_equation(ode, 2, dt)
plt.savefig("../figures/LV1.pdf",bbox_inches="tight")

In [ ]:
fig = solve_modified_equation(ode, 2:3, 0.11)
plt.xlim(0,8)
plt.savefig("../figures/LV2.pdf",bbox_inches="tight")

In [ ]:
fig = solve_modified_equation(ode, 2:4, 0.12)
plt.xlim(0,8)
plt.savefig("../figures/LV3.pdf",bbox_inches="tight")

In [ ]:
u0 = [1., 2.01]
tspan = (0.0, 66.4)
ode = ODEProblem(f, u0, tspan)

dt = 0.1
sol_euler = solve(ode, Euler(), dt=dt)
sol_ref = solve(ode, Tsit5())

# Explicit Euler method
A = @SArray [0//1;]
b = @SArray [1//1]
c = @SArray [0//1]

# Setup of symbolic variables
@variables dt_sym
u_sym = @variables p q
f_sym = similar(u_sym); f(f_sym, u_sym, nothing, nothing)

sol_euler = solve(ode, Euler(), dt=dt)

plt.scatter(last.(sol_euler.u), first.(sol_euler.u), label="Explicit Euler, dt = $dt", color="k")

for truncation_order in 2:6
    series = modified_equation(f_sym, u_sym, dt_sym, A, b, c, truncation_order)
    series = Symbolics.substitute.(series, dt_sym => dt)
    modified_f, _ = build_function(series, u_sym, expression=Val(false))
    modified_ode = ODEProblem((u, params, t) -> modified_f(u), ode.u0, tspan)
    modified_sol = solve(modified_ode, Tsit5(), abstol=1.0e-6, reltol=1.0e-6)
    plt.plot(last.(modified_sol.u), first.(modified_sol.u), label="Modified ODE, order $(truncation_order-1)",alpha=0.8)
end
plt.legend()
plt.xlabel(L"$q$"); plt.ylabel(L"$p$");
plt.savefig("../figures/LV4.pdf",bbox_inches="tight")

# Modifying integrators (Lotka-Volterra again)

This reproduces the example in Section 2.5.

In [ ]:
function f(du, u, params, t)
  p, q = u
  dp = (2 - q) * p
  dq = (p - 1) * q
  du[1] = dp; du[2] = dq
  return nothing
end

u0 = [1.5, 2.25]
tspan = (0.0, 15.0)
ode = ODEProblem(f, u0, tspan)

dt = 0.35
sol_euler = solve(ode, Euler(), dt=dt)
sol_ref = solve(ode, Tsit5(),saveat=0:0.1:15)

In [ ]:
plt.plot(last.(sol_ref.u), first.(sol_ref.u), label="Reference solution", color="k")
plt.scatter(last.(sol_euler.u), first.(sol_euler.u),
             label="Explicit Euler, dt = $dt")
plt.xlim(0.0, 4.0); plt.ylim(0.0, 2.5)
plt.legend()
plt.xlabel(L"$q$"); plt.ylabel(L"$p$");
plt.savefig("../figures/MI_LV1.pdf",bbox_inches="tight")

In [ ]:
# Explicit Euler method
A = @SArray [0//1;]
b = @SArray [1//1]
c = @SArray [0//1]

# Setup of symbolic variables
@variables dt_sym
u_sym = @variables p q
f_sym = similar(u_sym); f(f_sym, u_sym, nothing, nothing)

for truncation_order in 2:4
  series = modifying_integrator(f_sym, u_sym, dt_sym, A, b, c, truncation_order)
  series = Symbolics.substitute.(series, dt_sym => dt)
  modified_f, _ = build_function(series, u_sym, expression=Val(false))
  modified_ode = ODEProblem((u, params, t) -> modified_f(u), ode.u0, tspan)
  modified_sol_euler = solve(modified_ode, Euler(), dt=dt)
  plt.plot(last.(modified_sol_euler.u), first.(modified_sol_euler.u),
        label="Euler, modified ODE order $(truncation_order-1)")
end
plt.xlim(0.0, 4.0), plt.ylim(0.0, 2.5)
plt.legend()
plt.xlabel(L"$q$"); plt.ylabel(L"$p$");
plt.savefig("../figures/MI_LV2.pdf",bbox_inches="tight")